# Imports

In [1]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import json
import pandas as pd
# from selenium.webdriver.common.by import By

In [2]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [3]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Web Driver

In [4]:
chop = webdriver.ChromeOptions()
chop.add_extension('AdGuard AdBlocker 4.1.55.0.crx')
# driver = webdriver.Chrome('chromedriver', chrome_options=chop)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chop)

<ipython-input-4-a766d08505ca>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chop)
<ipython-input-4-a766d08505ca>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chop)


Getting All Year URLs

In [96]:
url = 'https://www.espncricinfo.com/cricketers/shakib-al-hasan-56143'
driver.get(url)
# data = page_soup.findAll(
#     "pre", {"style":"word-wrap: break-word; white-space: pre-wrap;"})

In [97]:
html = driver.page_source.encode('utf-8')
page_num = 0
content = driver.page_source
page_soup = soup(driver.page_source, 'lxml')

In [104]:
containers = page_soup.findAll(
    "a", {"class": "ds-flex ds-items-center ds-space-x-4"})
containers[0]['href'].split('/')[-1].split('-')[0]

'bangladesh'

In [5]:
all_year_t20i_links = {}

for year in range (2024, 2006, -1):
    link = "https://www.espncricinfo.com/records/year/team-match-results/" + str(year) + \
    "-" + str(year) + "/twenty20-internationals-3"
    all_year_t20i_links[year] = link

In [6]:
all_year_t20i_links

{2024: 'https://www.espncricinfo.com/records/year/team-match-results/2024-2024/twenty20-internationals-3',
 2023: 'https://www.espncricinfo.com/records/year/team-match-results/2023-2023/twenty20-internationals-3',
 2022: 'https://www.espncricinfo.com/records/year/team-match-results/2022-2022/twenty20-internationals-3',
 2021: 'https://www.espncricinfo.com/records/year/team-match-results/2021-2021/twenty20-internationals-3',
 2020: 'https://www.espncricinfo.com/records/year/team-match-results/2020-2020/twenty20-internationals-3',
 2019: 'https://www.espncricinfo.com/records/year/team-match-results/2019-2019/twenty20-internationals-3',
 2018: 'https://www.espncricinfo.com/records/year/team-match-results/2018-2018/twenty20-internationals-3',
 2017: 'https://www.espncricinfo.com/records/year/team-match-results/2017-2017/twenty20-internationals-3',
 2016: 'https://www.espncricinfo.com/records/year/team-match-results/2016-2016/twenty20-internationals-3',
 2015: 'https://www.espncricinfo.com/

Getting all match URLs, year wise

In [154]:
all_t20i_links = {}

for year, year_url in all_year_t20i_links.items():
    all_t20i_links[year] = []    
    driver.get(year_url)
    # data = page_soup.findAll(
    #     "pre", {"style":"word-wrap: break-word; white-space: pre-wrap;"})
    html = driver.page_source.encode('utf-8')
    page_num = 0
    content = driver.page_source
    page_soup = soup(driver.page_source, 'lxml')

    match_details_urls = page_soup.findAll(
    "a", {"class": "ds-inline-flex ds-items-start ds-leading-none"})
    match_dates = page_soup.findAll(
    "td", {"class": "ds-min-w-max ds-text-right ds-whitespace-nowrap"})[1:]
    match_count = 0
    match_details_json = {}

    for item in match_details_urls:
        if 'cricket-ground' in item['href']:            
            match_details_json['date'] = match_dates[match_count].text
            match_details_json['venue_link'] = item['href']
            match_details_json['venue_name'] = item['title']
        elif 'series' in item['href']:
            match_details_json['match_url'] = item['href']
            match_details_json['match_id'] = item.text.split('# ')[-1]
            all_t20i_links[year].append(match_details_json)
            match_details_json = {}
            match_count += 1 

    # match_urls = (lambda x: x['href'] for x in match_urls)
    # match_urls = list(map(lambda x: x['href'], match_urls))
    # match_urls = [x for x in match_urls if 'series/' in x]
    # all_t20i_links[year] = match_urls

In [161]:
remaining_matches ={}
remaining_matches[2024] = all_t20i_links[2024][:13]

Intialize JSON function

In [158]:
def initialize_json_values(table_code, json_obj):
    if('batsmen' in table_code):
        json_obj[table_code + 'left_handers'] = 0
        json_obj[table_code + 'left_hander_runs_scored'] = 0
        json_obj[table_code + 'left_hander_stamina'] = 0
        json_obj[table_code + 'left_hander_balls'] = 0
        json_obj[table_code + 'left_hander_boundaries'] = 0

        json_obj[table_code + 'right_handers'] = 0
        json_obj[table_code + 'right_hander_runs_scored'] = 0
        json_obj[table_code + 'right_hander_stamina'] = 0
        json_obj[table_code + 'right_hander_balls'] = 0
        json_obj[table_code + 'right_hander_boundaries'] = 0
    else:
        json_obj[table_code + 'left_hand_quicks'] = 0 
        json_obj[table_code + 'left_hand_quick_overs'] = 0
        json_obj[table_code + 'left_hand_quick_wickets'] = 0
        json_obj[table_code + 'left_hand_quick_dot_balls'] = 0
        json_obj[table_code + 'left_hand_quick_runs_conceded'] = 0
        json_obj[table_code + 'left_hand_quick_boundaries_conceded'] = 0
        json_obj[table_code + 'left_hand_quick_extras_conceded'] = 0

        json_obj[table_code + 'right_hand_quicks'] = 0 
        json_obj[table_code + 'right_hand_quick_overs'] = 0
        json_obj[table_code + 'right_hand_quick_wickets'] = 0
        json_obj[table_code + 'right_hand_quick_dot_balls'] = 0
        json_obj[table_code + 'right_hand_quick_runs_conceded'] = 0
        json_obj[table_code + 'right_hand_quick_boundaries_conceded'] = 0
        json_obj[table_code + 'right_hand_quick_extras_conceded'] = 0

        json_obj[table_code + 'spinners'] = 0 
        json_obj[table_code + 'spinner_overs'] = 0
        json_obj[table_code + 'spinner_wickets'] = 0
        json_obj[table_code + 'spinner_dot_balls'] = 0
        json_obj[table_code + 'spinner_runs_conceded'] = 0
        json_obj[table_code + 'spinner_boundaries_conceded'] = 0
        json_obj[table_code + 'spinner_extras_conceded'] = 0
    return json_obj

In [159]:
def getBowlingClutchMetric(metric, bowler_wickets, team_wickets_taken, team_run_rate_conceded, economy_rate):
    if metric == 'positive':
        calc_value = float(1 + float(bowler_wickets))/float(1 + float(team_wickets_taken)) * float(team_run_rate)/float(0.1 + float(economy_rate))
    elif metric == 'negative':
        calc_value = float(1 + float(team_wickets_taken))/float(1 + float(bowler_wickets)) * float(economy_rate)/float(0.1 + float(team_run_rate))
    return calc_value

def getBattingClutchMetric(metric, team_wickets_fallen, batting_strike_rate, team_run_rate):
    if metric == 'positive':
        if team_wickets_fallen == 0:
            calc_value = float(1) * float(batting_strike_rate/100) * float(team_run_rate)
        else:
            calc_value = float(team_wickets_fallen) * float((0.1 + float(batting_strike_rate))/100) * float(team_run_rate)
    elif metric == 'negative':
        calc_value = float(11-float(team_wickets_fallen)) * float(100/(100 + float(batting_strike_rate))) * float(team_run_rate)
    return calc_value

## FINAL FINAL CODE BELOW

In [162]:
data = []
spl_matches = []
skipped_matches = []
player_data_master = {}
summary_master = {}

In [12]:
t20i_links_sample = {}
t20i_links_sample[2020] = all_t20i_links[2024][:10]
# # t20i_links_sample[2020] = ['/series/west-indies-tour-of-new-zealand-2008-09-366700/new-zealand-vs-west-indies-2nd-t20i-366708/full-scorecard']
# t20i_links_sample[2020]

In [163]:
# odi_2022 = {}
# odi_2022[2022] = ['/series/ireland-in-usa-and-west-indies-2021-22-1291182/west-indies-vs-ireland-1st-odi-1277085/full-scorecard']

# data = []
# spl_matches = []
# player_data_master = {}
# summary_master ={}

world_cup_team_ids = [40,2,25,17,1,6,29,28,33,15,5,37,7,20,30,3,8,35,11,4]
espncricinfo = 'https://www.espncricinfo.com'

for year, match_details_list in remaining_matches.items():         
    print(year)
    for match_item in match_details_list:
        try:        
            scorecard_url = espncricinfo + match_item['match_url']
            live_score_url = scorecard_url.replace('full-scorecard','live-cricket-score')
            url = live_score_url

            driver.get(url)    
            html = driver.page_source.encode('utf-8')
            page_num = 0
            content = driver.page_source
            page_soup = soup(driver.page_source, 'lxml')
            
            json_obj = {}
            stage_count = 1
            print(stage_count)
            stage_count += 1    

            # try:
            json_obj['year'] = year
            json_obj['match_id'] = match_item['match_id']
            json_obj['espn_match_id'] = scorecard_url.split('/')[-2].split('-')[-1]        
            json_obj['scorecard_link'] = scorecard_url
            json_obj['live_score_link'] = scorecard_url.replace('full-scorecard','live-cricket-score')
            json_obj['match_overs_link'] = scorecard_url.replace('full-scorecard','match-overs-comparison')
            json_obj['tournament_link'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[0]['href']
            json_obj['tournament_id'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[0]['href'].split('-')[-1]
            json_obj['tournament'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[0].text
            json_obj['team1'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[1].text
            json_obj['team1_id'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[1]['href'].split('-')[-1]
            json_obj['team2'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[2].text
            json_obj['team2_id'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[2]['href'].split('-')[-1]
            if((int(json_obj['team2_id']) not in world_cup_team_ids) and (int(json_obj['team1_id']) not in world_cup_team_ids)):
                print('SKIP - ' + json_obj['match_id'] + ' - ' + json_obj['team1'] + ' vs ' + json_obj['team2'])
                skipped_matches.append(match_item)
                continue
            # json_obj['date'] = page_soup.findAll("span", {"class":"ds-text-tight-s ds-font-regular"})[4].text.split(' - ')[0]
            z = page_soup.findAll("div", {"class":"ds-text-tight-m ds-font-regular ds-text-typo-mid3"})
            json_obj['match_details'] = list(map(lambda x: x.text, z))
            # json_obj['date'] = ''.join(json_obj['match_details'][0].split(',')[2:4])[1:]
            json_obj['date'] = match_item['date']
            json_obj['venue_name'] = match_item['venue_name']
            json_obj['venue_link'] = match_item['venue_link']
            json_obj['venue_id'] = match_item['venue_link'].split('-')[-1]
            # json_obj['venue_name'] = json_obj['match_details'][0].split(',')[1][1:]
            # json_obj['day_or_night'] = page_soup.findAll("span", {"class":"ds-text-tight-s ds-font-regular"})[4].text.split(' - ')[1].split(' (')
            # json_obj['venue_link'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[19]['href']
            # json_obj['venue_id'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[19]['href'].split('-')[-1]
            # json_obj['full_venue_name'] = page_soup.findAll("a", {"class":"ds-inline-flex ds-items-start ds-leading-none"})[19].text
            print(json_obj['match_id'] + ' - ' + json_obj['team1'] + ' vs ' + json_obj['team2'])
            # except Exception as e:
            #     print(e)
            #     pass
            
            # try:
            url = scorecard_url        
            driver.get(url)
            print(url)    
            html = driver.page_source.encode('utf-8')
            page_num = 0
            content = driver.page_source
            page_soup = soup(driver.page_source, 'lxml')
            json_obj['match_result'] = page_soup.findAll("p", {"class":"ds-text-tight-s ds-font-medium ds-truncate ds-text-typo"})[0].text
            winner_string = page_soup.findAll("p", {"class":"ds-text-tight-s ds-font-medium ds-truncate ds-text-typo"})[0].text.split(' won')[0].replace('Match tied ', '').replace('(', '').replace('No result', 'x')
            winner_first_char = 'x' if winner_string == '' else winner_string[0]
            if(winner_first_char==json_obj['team1'][0]):
                json_obj['winner'] = json_obj['team1']
                json_obj['winner_id'] = json_obj['team1_id']
                json_obj['winning_team_1_or_2'] = 'team1'
            elif(winner_first_char==json_obj['team2'][0]):
                json_obj['winner'] = json_obj['team2']
                json_obj['winner_id'] = json_obj['team2_id']
                json_obj['winning_team_1_or_2'] = 'team2'
            else:
                json_obj['winner'] = 'None'
                json_obj['winner_id'] = -1
                json_obj['winning_team_1_or_2'] = 'None'
                continue          
            
            json_obj['team1_overs'] = page_soup.findAll("span", {"class":"ds-font-regular ds-text-tight-s"})[0].text.split(" ")[0]
            json_obj['team2_overs'] = page_soup.findAll("span", {"class":"ds-font-regular ds-text-tight-s"})[2].text.split(" ")[0]

            #team_scores = page_soup.findAll("td", {"class":"ds-font-bold ds-bg-fill-content-alternate ds-text-tight-m ds-min-w-max ds-text-right ds-text-typo"})
            # team_scores = page_soup.findAll("span", {"class":"ds-font-regular ds-text-tight-s"})
            team_scores = page_soup.findAll("div", {"class":"ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap"})
            json_obj['team1_runs'] = team_scores[0].strong.text.split(" ")[-1].split('/')[0]
            json_obj['team1_wickets'] = (team_scores[0].strong.text.split(" ")[-1].split('/')[1] if '/' in team_scores[0].strong.text else 10)
            json_obj['team1_runrate'] = page_soup.findAll("span", {"class":"ds-font-regular ds-text-tight-s"})[1].text.split(" ")[1][:-1]

            json_obj['team2_runs'] = team_scores[1].strong.text.split(" ")[-1].split('/')[0]
            json_obj['team2_wickets'] = (team_scores[1].strong.text.split(" ")[-1].split('/')[1] if '/' in team_scores[1].strong.text else 10)
            json_obj['team2_runrate'] = page_soup.findAll("span", {"class":"ds-font-regular ds-text-tight-s"})[3].text.split(" ")[1][:-1]

            print(stage_count) # 2
            stage_count += 1

            all_tables = page_soup.findAll("tbody")

            team1_batsmen = {}
            table_player_url = all_tables[0].findAll('a', attrs={'class': 'ds-inline-flex ds-items-start ds-leading-none'})
            table_player_url = list(map(lambda x: x['href'], table_player_url))
            table_player_url = [player for player in table_player_url if 'cricketer' in player]
            ball_details = all_tables[0].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right'})
            ball_details = list(map(lambda x: x.text, ball_details))
            ball_details = [ball.replace("-", "0.00") for ball in ball_details]
            runs_scored = all_tables[0].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right ds-text-typo'})
            runs_scored = list(map(lambda x: x.text, runs_scored))
            for i in range(len(runs_scored)):
                id = table_player_url[i].split('-')[-1]
                if (5*len(runs_scored)==len(ball_details)):
                    team1_batsmen[id] = {'player_url': table_player_url[i], 'runs': runs_scored[i], 'balls': ball_details[5*i], \
                                    'minutes': ball_details[5*i+1], 'boundaries': int(float(ball_details[5*i+2])) + int(float(ball_details[5*i+3])),\
                                        'strike_rate': float(ball_details[5*i+4])}
                elif (6*len(runs_scored)==len(ball_details) and '' in ball_details):
                    team1_batsmen[id] = {'player_url': table_player_url[i], 'runs': runs_scored[i], 'balls': ball_details[6*i], \
                                    'minutes': ball_details[6*i+1], 'boundaries': int(float(ball_details[6*i+2])) + int(float(ball_details[6*i+3])),\
                                        'strike_rate': float(ball_details[6*i+4])}
                else:
                    team1_batsmen[id] = {'player_url': table_player_url[i], 'runs': runs_scored[i], 'balls': ball_details[4*i], \
                                    'minutes': -1, 'boundaries': int(float(ball_details[4*i+1])) + int(float(ball_details[4*i+2])),\
                                        'strike_rate': float(ball_details[4*i+3])}
            json_obj['team1_batters_used'] = len(runs_scored)


            team2_batsmen = {}
            table_player_url = all_tables[2].findAll('a', attrs={'class': 'ds-inline-flex ds-items-start ds-leading-none'})
            table_player_url = list(map(lambda x: x['href'], table_player_url))
            table_player_url = [player for player in table_player_url if 'cricketer' in player]
            ball_details = all_tables[2].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right'})
            ball_details = list(map(lambda x: x.text, ball_details))
            ball_details = [ball.replace("-", "0.00") for ball in ball_details]
            runs_scored = all_tables[2].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right ds-text-typo'})
            runs_scored = list(map(lambda x: x.text, runs_scored))
            for i in range(len(runs_scored)):
                id = table_player_url[i].split('-')[-1]
                if (5*len(runs_scored)==len(ball_details)):
                    team2_batsmen[id] = {'player_url': table_player_url[i], 'runs': runs_scored[i], 'balls': ball_details[5*i], \
                                    'minutes': ball_details[5*i+1], 'boundaries': int(float(ball_details[5*i+2])) + int(float(ball_details[5*i+3])),\
                                        'strike_rate': float(ball_details[5*i+4])}
                elif (6*len(runs_scored)==len(ball_details) and '' in ball_details):
                    team2_batsmen[id] = {'player_url': table_player_url[i], 'runs': runs_scored[i], 'balls': ball_details[6*i], \
                                    'minutes': ball_details[6*i+1], 'boundaries': int(float(ball_details[6*i+2])) + int(float(ball_details[6*i+3])),\
                                        'strike_rate': float(ball_details[6*i+4])}
                else:
                    team2_batsmen[id] = {'player_url': table_player_url[i], 'runs': runs_scored[i], 'balls': ball_details[4*i], \
                                    'minutes': -1, 'boundaries': int(float(ball_details[4*i+1])) + int(float(ball_details[4*i+2])),\
                                        'strike_rate': float(ball_details[4*i+3])}
            json_obj['team2_batters_used'] = len(runs_scored)

            print(stage_count) # 3
            stage_count += 1

            team1_bowlers = {}
            table_player_url = all_tables[3].findAll('a', attrs={'class': 'ds-inline-flex ds-items-start ds-leading-none'})
            table_player_url = list(map(lambda x: x['href'], table_player_url))
            table_player_url = [player for player in table_player_url if 'cricketer' in player]
            ball_details = all_tables[3].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right'})
            ball_details = list(map(lambda x: x.text, ball_details))
            ball_details = [ball.replace("-", "0.00") for ball in ball_details]
            wickets_taken = all_tables[3].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-text-right'})
            wickets_taken = list(map(lambda x: x.text, wickets_taken))
            print(ball_details)
            for i in range(len(wickets_taken)):
                id = table_player_url[i].split('-')[-1]
                if (9*len(wickets_taken)==len(ball_details)):
                    team1_bowlers[id] = {'player_url': table_player_url[i], 'wickets': wickets_taken[i], 'overs': ball_details[9*i], \
                                    'runs_conceded': ball_details[9*i+2],'economy':ball_details[9*i+3], 'dots': ball_details[9*i+4], \
                                    'boundaries_conceded': int(float(ball_details[9*i+5])) + int(float(ball_details[9*i+6])),\
                                    'extras_conceded': int(float(ball_details[9*i+7])) + int(float(ball_details[9*i+8]))}
                elif (10*len(wickets_taken)==len(ball_details) and '' in ball_details):
                    team1_bowlers[id] = {'player_url': table_player_url[i], 'wickets': wickets_taken[i], 'overs': ball_details[10*i], \
                                    'runs_conceded': ball_details[10*i+2],'economy':ball_details[10*i+3], 'dots': ball_details[10*i+4], \
                                    'boundaries_conceded': int(float(ball_details[10*i+5])) + int(float(ball_details[10*i+6])),\
                                    'extras_conceded': int(float(ball_details[10*i+7])) + int(float(ball_details[10*i+8]))}
                else:
                    team1_bowlers[id] = {'player_url': table_player_url[i], 'wickets': wickets_taken[i], 'overs': ball_details[6*i], \
                                    'runs_conceded': ball_details[6*i+2],'economy':ball_details[6*i+3], 'dots': -1, \
                                    'boundaries_conceded': -1, \
                                    'extras_conceded': int(float(ball_details[6*i+4])) + int(float(ball_details[6*i+5]))}
            json_obj['team1_bowlers_used'] = len(wickets_taken)
                
            team2_bowlers = {}
            table_player_url = all_tables[1].findAll('a', attrs={'class': 'ds-inline-flex ds-items-start ds-leading-none'})
            table_player_url = list(map(lambda x: x['href'], table_player_url))
            table_player_url = [player for player in table_player_url if 'cricketer' in player]
            ball_details = all_tables[1].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right'})
            ball_details = list(map(lambda x: x.text, ball_details))
            ball_details = [ball.replace("-", "0.00") for ball in ball_details]
            wickets_taken = all_tables[1].findAll('td', attrs={'class': 'ds-w-0 ds-whitespace-nowrap ds-text-right'})
            wickets_taken = list(map(lambda x: x.text, wickets_taken))
            for i in range(len(wickets_taken)):
                id = table_player_url[i].split('-')[-1]
                if (9*len(wickets_taken)==len(ball_details)):
                    team2_bowlers[id] = {'player_url': table_player_url[i], 'wickets': wickets_taken[i], 'overs': ball_details[9*i], \
                                    'runs_conceded': ball_details[9*i+2],'economy':ball_details[9*i+3], 'dots': ball_details[9*i+4], \
                                    'boundaries_conceded': int(float(ball_details[9*i+5])) + int(float(ball_details[9*i+6])),\
                                    'extras_conceded': int(float(ball_details[9*i+7])) + int(float(ball_details[9*i+8]))}
                elif (10*len(wickets_taken)==len(ball_details) and '' in ball_details):
                    team2_bowlers[id] = {'player_url': table_player_url[i], 'wickets': wickets_taken[i], 'overs': ball_details[10*i], \
                                    'runs_conceded': ball_details[10*i+2],'economy':ball_details[10*i+3], 'dots': ball_details[10*i+4], \
                                    'boundaries_conceded': int(float(ball_details[10*i+5])) + int(float(ball_details[10*i+6])),\
                                    'extras_conceded': int(float(ball_details[10*i+7])) + int(float(ball_details[10*i+8]))}
                else:
                    team2_bowlers[id] = {'player_url': table_player_url[i], 'wickets': wickets_taken[i], 'overs': ball_details[6*i], \
                                    'runs_conceded': ball_details[6*i+2],'economy':ball_details[6*i+3], 'dots': -1, \
                                    'boundaries_conceded': -1, \
                                    'extras_conceded': int(float(ball_details[6*i+4])) + int(float(ball_details[6*i+5]))}
            json_obj['team2_bowlers_used'] = len(wickets_taken)
            print(stage_count) # 4
            stage_count += 1

            database_count = 0            
            json_obj['team1_batsmen_clutch_metric'] = 0
            json_obj['team2_batsmen_clutch_metric'] = 0
            json_obj['team1_bowlers_clutch_metric'] = 0
            json_obj['team2_bowlers_clutch_metric'] = 0
            
            for table_data in [team1_batsmen, team1_bowlers, team2_batsmen, team2_bowlers]:
                database_count += 1
                
                for player_id, player_performance in table_data.items():        
                        if player_id not in player_data_master.keys():
                            print('Record not available for ', player_id)
                            player_data_master[player_id] = {}
                            url = espncricinfo + player_performance['player_url']
                            driver.get(url)    
                            html = driver.page_source.encode('utf-8')
                            page_num = 0
                            content = driver.page_source
                            page_soup = soup(driver.page_source, 'lxml')

                            team_names = page_soup.findAll("a", {"class": "ds-flex ds-items-center ds-space-x-4"})
                            player_data_master[player_id]['national_team_name'] = team_names[0]['href'].split('/')[-1].split('-')[0]
                            player_data_master[player_id]['national_team_id'] = team_names[0]['href'].split('/')[-1].split('-')[-1]

                            player_detail_name = page_soup.findAll("p", {"class":"ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3"})
                            player_detail_name = list(map(lambda x: x.text.lower(), player_detail_name))
                            player_detail_value = page_soup.findAll("span", {"class":"ds-text-title-s ds-font-bold ds-text-typo"})
                            player_detail_value = list(map(lambda x: x.text.lower(), player_detail_value))
                            

                            for i in range(len(player_detail_name)):
                                player_data_master[player_id][player_detail_name[i]] = player_detail_value[i]
                                table_data[player_id][player_detail_name[i]] = player_detail_value[i]
                            print('Added record for '+ player_id + " " + player_data_master[player_id]['full name'])

                        else:
                            print('Record already available')
                            player_details = player_data_master[player_id]
                            player_detail_name = list(player_details.keys())
                            player_detail_value = list(player_details.values())

                            for i in range(len(player_detail_name)):
                                table_data[player_id][player_detail_name[i]] = player_detail_value[i]
                            print('Done for '+ player_id + " " + player_data_master[player_id]['full name'])
                        
                        #################### Calculating clutch metric #######################
                        if (json_obj['winner_id'] != -1):
                            print('National team ID, winner ID', player_data_master[player_id]['national_team_id'], json_obj['winner_id'])

                            # Checking if the player's contribution is from batting or bowling
                            if (database_count%2): # Batting

                                # Checking if the player's team won
                                if player_data_master[player_id]['national_team_id'] == json_obj['winner_id']:                               
                                    # Batting clutch metric
                                    winning_teamcode = json_obj['winning_team_1_or_2']
                                    teamcode = winning_teamcode # player's team has won
                                    team_run_rate = json_obj[teamcode + '_runrate']
                                    team_wickets_fallen = json_obj[teamcode + '_wickets']
                                    batting_strike_rate = player_performance['strike_rate']

                                    if (str(year) + '_pos_batting_clutch_metric') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_pos_batting_clutch_metric'] = 0
                                    
                                    if (str(year) + '_matches_played') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_matches_played'] = 0
                                    
                                    print('positive - batting', team_wickets_fallen, batting_strike_rate, team_run_rate)
                                    calc_value = getBattingClutchMetric('positive', float(team_wickets_fallen), float(batting_strike_rate), float(team_run_rate))
                                    player_data_master[player_id][str(year) + '_pos_batting_clutch_metric'] += calc_value
                                    json_obj[teamcode + '_batsmen_clutch_metric'] += calc_value
                                
                                # Checking if the player's team lost
                                elif json_obj['winner_id'] != -1: 

                                    # Batting clutch metric
                                    winning_teamcode = json_obj['winning_team_1_or_2']
                                    teamcode = 'team1' if winning_teamcode=='team2' else 'team2'
                                    team_run_rate = json_obj[teamcode + '_runrate']
                                    team_wickets_fallen = json_obj[teamcode + '_wickets']
                                    batting_strike_rate = player_performance['strike_rate']     

                                    if (str(year) + '_neg_batting_clutch_metric') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_neg_batting_clutch_metric'] = 0

                                    if (str(year) + '_matches_played') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_matches_played'] = 0

                                    print('negative - batting', team_wickets_fallen, batting_strike_rate, team_run_rate)
                                    calc_value = getBattingClutchMetric('negative', float(team_wickets_fallen), float(batting_strike_rate), float(team_run_rate))
                                    player_data_master[player_id][str(year) + '_neg_batting_clutch_metric'] -= calc_value
                                    json_obj[teamcode + '_batsmen_clutch_metric'] -= calc_value
                                    # print(teamcode,'_batsmen_clutch_metric', json_obj[teamcode + '_batsmen_clutch_metric'])
                                print('getBattingClutchMetric', calc_value)
                                print(teamcode + '_batsmen_clutch_metric', json_obj[teamcode + '_batsmen_clutch_metric'])
                            
                            else: # Bowling
                                
                                if player_data_master[player_id]['national_team_id'] == json_obj['winner_id']:                               
                                    # Bowling clutch metric
                                    winning_teamcode = json_obj['winning_team_1_or_2']
                                    opposition_teamcode = 'team1' if winning_teamcode=='team2' else 'team2'
                                    bowler_wickets = player_performance['wickets']
                                    team_wickets_taken = json_obj[opposition_teamcode + '_wickets']
                                    team_run_rate_conceded = json_obj[opposition_teamcode + '_runrate']
                                    economy_rate = player_performance['economy']

                                    if (str(year) + '_pos_bowling_clutch_metric') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_pos_bowling_clutch_metric'] = 0  

                                    if (str(year) + '_matches_played') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_matches_played'] = 0 

                                    print('positive - bowling', bowler_wickets, team_wickets_taken, team_run_rate_conceded, economy_rate)
                                    calc_value = getBowlingClutchMetric('positive', float(bowler_wickets), float(team_wickets_taken), float(team_run_rate_conceded), float(economy_rate))
                                    player_data_master[player_id][str(year) + '_pos_bowling_clutch_metric'] += calc_value
                                    json_obj[teamcode + '_bowlers_clutch_metric'] += calc_value
                                
                                # Checking if the player's team lost
                                elif json_obj['winner_id'] != -1: 

                                    # Bowling clutch metric
                                    winning_teamcode = json_obj['winning_team_1_or_2']
                                    opposition_teamcode = winning_teamcode
                                    bowler_wickets = player_performance['wickets']
                                    team_wickets_taken = json_obj[opposition_teamcode + '_wickets']
                                    team_run_rate_conceded = json_obj[opposition_teamcode + '_runrate']
                                    economy_rate = player_performance['economy']

                                    if (str(year) + '_neg_bowling_clutch_metric') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_neg_bowling_clutch_metric'] = 0

                                    if (str(year) + '_matches_played') not in player_data_master[player_id].keys():
                                        player_data_master[player_id][str(year) + '_matches_played'] = 0 

                                    print('negative - bowling', bowler_wickets, team_wickets_taken, team_run_rate_conceded, economy_rate)
                                    calc_value = getBowlingClutchMetric('negative', float(bowler_wickets), float(team_wickets_taken), float(team_run_rate_conceded), float(economy_rate))
                                    player_data_master[player_id][str(year) + '_neg_bowling_clutch_metric'] -= calc_value
                                    json_obj[teamcode + '_bowlers_clutch_metric'] -= calc_value      

                                print('getBowlingClutchMetric', calc_value)                      
                                print(teamcode + '_bowlers_clutch_metric', json_obj[teamcode + '_bowlers_clutch_metric'])

                            player_data_master[player_id][str(year) + '_matches_played'] += 1

            print('Clutch metric calculation done')            
            print(stage_count) # 5
            stage_count += 1
            
            # json_obj ={}
            match_player_data = {'team1_batsmen_': team1_batsmen, 'team2_batsmen_': team2_batsmen, 
                                'team1_bowlers_': team1_bowlers, 'team2_bowlers_': team2_bowlers}

            print(stage_count) # 6
            stage_count += 1

            for table_code, table_data in match_player_data.items():
                json_obj = initialize_json_values(table_code, json_obj)
                json_data = list(table_data.values())

                for player in json_data:
                    if('batsmen' in table_code):
                        if('runs' in player.keys()):
                            # Types of Batsmen
                            if('left' in player['batting style'].lower()):
                                # print('left - ', player)
                                json_obj[table_code + 'left_handers'] += 1
                                json_obj[table_code + 'left_hander_runs_scored'] += int(player['runs'])
                                json_obj[table_code + 'left_hander_stamina'] += int(player['minutes'])
                                json_obj[table_code + 'left_hander_balls'] += int(player['balls'])
                                json_obj[table_code + 'left_hander_boundaries'] += int(player['boundaries'])                    
                            else:
                                # print('right - ', player)
                                json_obj[table_code + 'right_handers'] += 1
                                json_obj[table_code + 'right_hander_runs_scored'] += int(player['runs'])
                                json_obj[table_code + 'right_hander_stamina'] += int(player['minutes'])
                                json_obj[table_code + 'right_hander_balls'] += int(player['balls'])
                                json_obj[table_code + 'right_hander_boundaries'] += int(player['boundaries'])   

                    if('bowler' in table_code):
                        if('wickets' in list(player.keys())):
                            # Types of Bowlers
                            if(('left' in player['bowling style'].lower()) and (('fast' in player['bowling style'].lower()) or ('medium' in player['bowling style'].lower()))):
                                # print('leftquick - ', player)
                                json_obj[table_code + 'left_hand_quicks'] += 1
                                json_obj[table_code + 'left_hand_quick_overs'] += float(player['overs'])
                                json_obj[table_code + 'left_hand_quick_wickets'] += int(player['wickets'])
                                json_obj[table_code + 'left_hand_quick_dot_balls'] += int(player['dots'])
                                json_obj[table_code + 'left_hand_quick_runs_conceded'] += int(player['runs_conceded'])
                                json_obj[table_code + 'left_hand_quick_boundaries_conceded'] += int(player['boundaries_conceded'])
                                json_obj[table_code + 'left_hand_quick_extras_conceded'] += int(player['extras_conceded'])                                      
                            if(('right' in player['bowling style'].lower()) and (('fast' in player['bowling style'].lower()) or ('medium' in player['bowling style'].lower()))):
                                # print('rightquick - ', player)
                                json_obj[table_code + 'right_hand_quicks'] += 1
                                json_obj[table_code + 'right_hand_quick_overs'] += float(player['overs'])
                                json_obj[table_code + 'right_hand_quick_wickets'] += int(player['wickets'])
                                json_obj[table_code + 'right_hand_quick_dot_balls'] += int(player['dots'])
                                json_obj[table_code + 'right_hand_quick_runs_conceded'] += int(player['runs_conceded'])
                                json_obj[table_code + 'right_hand_quick_boundaries_conceded'] += int(player['boundaries_conceded'])  
                                json_obj[table_code + 'right_hand_quick_extras_conceded'] += int(player['extras_conceded'])                                      
                            if(('spin' in player['bowling style'].lower()) or ('wrist' in player['bowling style'].lower()) or ('leg' in player['bowling style'].lower()) | \
                            ('orthodox' in player['bowling style'].lower()) or ('off' in player['bowling style'].lower()) or ('china' in player['bowling style'].lower())):
                                # print('spin - ', player)
                                json_obj[table_code + 'spinners'] += 1
                                json_obj[table_code + 'spinner_overs'] += float(player['overs'])
                                json_obj[table_code + 'spinner_wickets'] += int(player['wickets'])
                                json_obj[table_code + 'spinner_dot_balls'] += int(player['dots'])
                                json_obj[table_code + 'spinner_runs_conceded'] += int(player['runs_conceded'])
                                json_obj[table_code + 'spinner_boundaries_conceded'] += int(player['boundaries_conceded'])
                                json_obj[table_code + 'spinner_extras_conceded'] += int(player['extras_conceded'])

            print("Done compiling stats of left/right, spin/fast matchups")
            print(json_obj)
            z =  all_tables[0].findAll('a', attrs={'class': 'ds-inline-flex ds-items-start ds-leading-none'})
            json_obj['team1_player_ids'] = list(map(lambda x: x['href'].split('-')[-1], z))

            z =  all_tables[2].findAll('a', attrs={'class': 'ds-inline-flex ds-items-start ds-leading-none'})
            json_obj['team2_player_ids'] = list(map(lambda x: x['href'].split('-')[-1], z))

            print(stage_count) # 7
            stage_count += 1

            for team_code in ['team1_', 'team2_']:
                json_obj[team_code + 'batsmen_count'] = 0
                json_obj[team_code + 'bowler_count'] = 0
                json_obj[team_code + 'allrounder_count'] = 0

                for i in range(11):
                    try:
                        team_player_data = player_data_master[json_obj[team_code + 'player_ids'][i]]
                    except:
                        json_obj[team_code + 'allrounder_count'] += 1
                        continue
                    
                    try:
                        if('bat' in team_player_data['playing role']):
                            json_obj[team_code + 'batsmen_count'] += 1
                        elif('bowl' in team_player_data['playing role']):
                            json_obj[team_code + 'bowler_count'] += 1
                        else:
                            json_obj[team_code + 'allrounder_count'] += 1
                    except:
                        continue

            print(stage_count) # 8
            stage_count += 1

            # Getting Overwise data    

            url = json_obj['match_overs_link']
            id = json_obj['match_id']
            try:
                team1_overs = int(float(json_obj['team1_overs']))
            except:
                team1_overs = 0
            try:
                team2_overs = int(float(json_obj['team2_overs']))
            except: 
                team2_overs = 0
            print(team1_overs, team2_overs)
            driver.get(url)
            page_num = 0
            content = driver.page_source
            page_soup = soup(driver.page_source, 'lxml')

            over_details = page_soup.findAll("span", {"class": "ds-text-tight-s ds-font-regular ds-ml-1 ds-text-typo-mid3"})
            all_over_count = 1
            team1_ocount = 1
            team2_ocount = 1
            team1_runs = []
            team1_wickets = []
            team2_runs = []
            team2_wickets = []

            print(stage_count) # 9
            stage_count += 1

            over_details = page_soup.findAll("span", {"class": "ds-text-tight-s ds-font-regular ds-ml-1 ds-text-typo-mid2"})
            all_over_count = 1
            team1_ocount = 1
            team2_ocount = 1
            team1_runs = []
            team1_wickets = []
            team2_runs = []
            team2_wickets = []

            if(len(json_obj['team1_overs'].split('.'))==2):
                extra_one = 1
            else:
                extra_one = 0

            if(len(json_obj['team2_overs'].split('.'))==2):
                extra_two = 1
            else:
                extra_two = 0

            try:
                team1_overs = min(int(float(json_obj['team1_overs'])) + extra_one, 50)
            except:
                team1_overs = 0
            try:
                team2_overs = min(int(float(json_obj['team2_overs'])) + extra_two, 50)
            except: 
                team2_overs = 0

            driver.get(url)
            # print("Fetched Page", id)
            page_num = 0
            content = driver.page_source
            page_soup = soup(driver.page_source, 'lxml')
            print(len(over_details))
            team1_excess_flag = False
            team2_excess_flag = False

            for over in over_details:
                # print('run ', str(all_over_count), '/', len(over_details))
                over_desc = over.text.replace("<!-- -->", "").replace("(", "").replace(")", "")
                if ((((all_over_count%2) == 0) or (team1_excess_flag)) and (team2_ocount <= team2_overs)):
                    team2_runs.append(over_desc.split(" ")[0])
                    team2_wickets.append(over_desc.split(" ")[2])
                    # print('team2 ', team2_ocount)
                    team2_ocount += 1
                    # if(team2_ocount > team2_overs):
                    #     team2_excess_flag = False
                    
                elif (team1_ocount <= team1_overs):
                    team1_runs.append(over_desc.split(" ")[0])
                    team1_wickets.append(over_desc.split(" ")[2])
                    # print('team1 ', team1_ocount)
                    team1_ocount += 1
                    if(team1_ocount > team1_overs):
                        team1_excess_flag = True            
                    
                all_over_count += 1

            # team1_runs = [eval(i) for i in team1_runs]
            # team2_runs = [eval(i) for i in team2_runs]
            # team1_wickets = [eval(i) for i in team1_wickets]
            # team2_wickets = [eval(i) for i in team2_wickets]

            print('Actual Overs', json_obj['team1_overs'], json_obj['team2_overs'])
            print('team1_overs',team1_overs, 'team2_overs', team2_overs)
            print('team1_runs',team1_runs)
            print('team2_runs',team2_runs)
            print('team1_wickets',team1_wickets)
            print('team2_wickets',team2_wickets)
            team1_pp_runs_scored = team1_middle_runs_scored = team1_death_runs_scored = 0
            team1_pp_wickets_lost = team1_middle_wickets_lost = team1_death_wickets_lost = 0

            team2_pp_runs_scored = team2_middle_runs_scored = team2_death_runs_scored = 0
            team2_pp_wickets_lost = team2_middle_wickets_lost = team2_death_wickets_lost = 0

            team1_ocount = 1
            team2_ocount = 1

            for i in range(20):
                if i<6:
                    if (team1_ocount <= team1_overs):
                        team1_pp_runs_scored += int(team1_runs[i])
                        team1_pp_wickets_lost += int(team1_wickets[i])
                        team1_ocount += 1
                    if (team2_ocount <= team2_overs):
                        team2_pp_runs_scored += int(team2_runs[i])
                        team2_pp_wickets_lost += int(team2_wickets[i])
                        team2_ocount += 1
                elif (6<=i<=14):
                    if (team1_ocount <= team1_overs):
                        team1_middle_runs_scored += int(team1_runs[i])
                        team1_middle_wickets_lost += int(team1_wickets[i])
                        team1_ocount += 1
                    if (team2_ocount <= team2_overs):
                        team2_middle_runs_scored += int(team2_runs[i])
                        team2_middle_wickets_lost += int(team2_wickets[i])
                        team2_ocount += 1
                elif i>14:
                    if (team1_ocount <= team1_overs):
                        team1_death_runs_scored += int(team1_runs[i])
                        team1_death_wickets_lost += int(team1_wickets[i])
                        team1_ocount += 1
                    if (team2_ocount <= team2_overs):
                        team2_death_runs_scored += int(team2_runs[i])
                        team2_death_wickets_lost += int(team2_wickets[i])
                        team2_ocount += 1

            json_obj['team1_pp_runs_scored'] = team1_pp_runs_scored 
            json_obj['team1_middle_runs_scored'] = team1_middle_runs_scored
            json_obj['team1_death_runs_scored'] = team1_death_runs_scored

            if(team1_overs>0):
                json_obj['team1_pp_run_rate'] = team1_pp_runs_scored/min(team1_overs, 6)
            else:
                json_obj['team1_pp_run_rate'] = 0
            if(team1_overs>6):
                json_obj['team1_middle_run_rate'] = team1_middle_runs_scored/min(team1_overs-6, 9)
            else:
                json_obj['team1_middle_run_rate'] = 0
            if(team2_overs>14):
                json_obj['team1_death_run_rate'] = team1_death_runs_scored/min(team1_overs-14, 5)
            else:
                json_obj['team1_death_run_rate'] = 0

            json_obj['team1_pp_wickets_lost'] = team1_pp_wickets_lost
            json_obj['team1_middle_wickets_lost'] = team1_middle_wickets_lost
            json_obj['team1_death_wickets_lost'] = team1_death_wickets_lost

            json_obj['team2_pp_runs_scored'] = team2_pp_runs_scored
            json_obj['team2_middle_runs_scored'] = team2_middle_runs_scored
            json_obj['team2_death_runs_scored'] = team2_death_runs_scored

            if(team2_overs>0):
                json_obj['team2_pp_run_rate'] = team2_pp_runs_scored/min(team2_overs, 6)
            else:
                json_obj['team2_pp_run_rate']
            if(team2_overs>6):
                json_obj['team2_middle_run_rate'] = team2_middle_runs_scored/min(team2_overs-6, 9)
            else:
                json_obj['team2_middle_run_rate'] = 0
            if(team2_overs>14):
                json_obj['team2_death_run_rate'] = team2_death_runs_scored/min(team2_overs-14, 5)
            else:
                json_obj['team2_death_run_rate'] = 0

            json_obj['team2_pp_wickets_lost'] = team2_pp_wickets_lost
            json_obj['team2_middle_wickets_lost'] = team2_middle_wickets_lost
            json_obj['team2_death_wickets_lost'] = team2_death_wickets_lost

            print(stage_count) # 10
            stage_count += 1

            data.append(json_obj)
            print("Match", len(data), "Over-wise done")
            print(json_obj)
    
        except Exception as e:
            # raise Exception(e)
            spl_match = {}
            spl_match['id'] = id
            spl_match['url'] = url
            spl_match['stage'] = stage_count
            spl_match['error'] = e
            spl_match['match_details_list'] = match_details_list
            spl_matches.append(spl_match)
            print('--ERROR--')
            print(spl_match)
            pass
        
    match_pd = pd.DataFrame.from_dict(data)
    summary_master[year] = match_pd
    match_pd.to_csv(str(year) + '.csv', index=False)

2024
1
2722 - Afghanistan vs Bangladesh
https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2024-1411166/afghanistan-vs-bangladesh-52nd-match-super-eights-group-1-1415752/full-scorecard
2
3
['3.5', '0', '26', '6.78', '12', '3', '1', '0', '0', '', '2', '0', '15', '7.50', '3', '1', '0', '2', '0', '', '2', '0', '15', '7.50', '5', '2', '0', '0', '0', '', '4', '0', '23', '5.75', '9', '2', '0', '1', '0', '', '4', '0', '13', '3.25', '18', '2', '0', '0', '0', '', '2', '0', '5', '2.50', '7', '0', '0', '0', '0', '']
4
Record not available for  974087
Added record for 974087 rahmanullah gurbaz
National team ID, winner ID 40 40
positive - batting 5 78.18 5.75
getBattingClutchMetric 22.5055
team1_batsmen_clutch_metric 22.5055
Record not available for  921509
Added record for 921509 ibrahim zadran
National team ID, winner ID 40 40
positive - batting 5 62.06 5.75
getBattingClutchMetric 17.871000000000002
team1_batsmen_clutch_metric 40.37650000000001
Record not available for  819429
Added reco

In [165]:
# match_pd = match_pd.iloc[::-1].reset_index(drop=True)
# match_pd.to_csv(str(year) + '.csv', index=False)

In [33]:
summary_master.keys()

dict_keys([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007])

In [64]:
# first_time = True
# for year, year_matches_df in summary_master.items():
#     print(len(year_matches_df))
#     if (first_time):
#         all_matches_t20_wc_df = year_matches_df
#         first_time = False
#         print('yin')
#     else:
#         all_matches_t20_wc_df = pd.concat([all_matches_t20_wc_df, year_matches_df])
#         print('yang')
summary_master[2007].to_csv('all_matches_data_t20_wc.csv', index=False)

In [65]:
len(summary_master[2007])

1483

In [231]:
player_data_master_old = pd.read_csv('player_data_master_t20_wc.csv')
player_data_master_old = player_data_master_old.convert_dtypes()
player_data_master_old['player_id'] = player_data_master_old['player_id'].astype(str)
player_data_master_old.head(2)

,player_id,national_team_name,national_team_id,full name,born,age,batting style,playing role,2024_neg_batting_clutch_metric,2024_matches_played,2024_pos_batting_clutch_metric,2023_pos_batting_clutch_metric,2023_matches_played,2023_neg_batting_clutch_metric,2022_neg_batting_clutch_metric,2022_matches_played,2022_pos_batting_clutch_metric,2021_neg_batting_clutch_metric,2021_matches_played,2020_neg_batting_clutch_metric,2020_matches_played,2020_pos_batting_clutch_metric,2019_neg_batting_clutch_metric,2019_matches_played,2019_pos_batting_clutch_metric,fielding position,2016_neg_batting_clutch_metric,2016_matches_played,2016_pos_batting_clutch_metric,2015_pos_batting_clutch_metric,2015_matches_played,2015_neg_batting_clutch_metric,2014_neg_batting_clutch_metric,2014_matches_played,2013_neg_batting_clutch_metric,2013_matches_played,2013_pos_batting_clutch_metric,2012_pos_batting_clutch_metric,2012_matches_played,2012_neg_batting_clutch_metric,2011_pos_batting_clutch_metric,2011_matches_played,2011_neg_batting_clutch_metric,bowling style,2021_pos_batting_clutch_metric,2018_pos_batting_clutch_metric,2018_matches_played,2018_neg_batting_clutch_metric,2024_neg_bowling_clutch_metric,2023_neg_bowling_clutch_metric,2022_neg_bowling_clutch_metric,2022_pos_bowling_clutch_metric,2020_neg_bowling_clutch_metric,2020_pos_bowling_clutch_metric,2018_pos_bowling_clutch_metric,2018_neg_bowling_clutch_metric,2017_neg_batting_clutch_metric,2017_matches_played,2017_pos_batting_clutch_metric,2017_pos_bowling_clutch_metric,2024_pos_bowling_clutch_metric,2023_pos_bowling_clutch_metric,2021_neg_bowling_clutch_metric,2021_pos_bowling_clutch_metric,2016_pos_bowling_clutch_metric,2016_neg_bowling_clutch_metric,2015_neg_bowling_clutch_metric,2015_pos_bowling_clutch_metric,2014_pos_batting_clutch_metric,2014_pos_bowling_clutch_metric,2014_neg_bowling_clutch_metric,2013_pos_bowling_clutch_metric,2012_pos_bowling_clutch_metric,2012_neg_bowling_clutch_metric,2011_neg_bowling_clutch_metric,2011_pos_bowling_clutch_metric,2019_pos_bowling_clutch_metric,education,also known as,nicknames,height,2017_neg_bowling_clutch_metric,2019_neg_bowling_clutch_metric,2009_neg_batting_clutch_metric,2009_matches_played,2009_neg_bowling_clutch_metric,2009_pos_bowling_clutch_metric,2010_pos_batting_clutch_metric,2010_matches_played,2013_neg_bowling_clutch_metric,2010_neg_batting_clutch_metric,2010_neg_bowling_clutch_metric,2010_pos_bowling_clutch_metric,2008_pos_batting_clutch_metric,2008_matches_played,2008_pos_bowling_clutch_metric,2008_neg_bowling_clutch_metric,2008_neg_batting_clutch_metric,2009_pos_batting_clutch_metric,2007_neg_batting_clutch_metric,2007_matches_played,2007_neg_bowling_clutch_metric,2007_pos_batting_clutch_metric,2007_pos_bowling_clutch_metric,other,died
0,670035,west,4,brandon alexander king,"december 16, 1994, kingston, jamaica",29y 187d,right hand bat,top order batter,-48.514704,10,505.677721,701.316028,13,-121.74086,-173.538350,17,301.907971,-35.161545,3,-36.165217,4,108.877682,-67.71203,5,41.041,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>
1,333066,west,4,johnson charles,"january 14, 1989, st lucia",35y 158d,right hand bat,wicketkeeper batter,-47.562383,11,310.139647,293.131878,7,-47.39507,-35.783941,4,66.991050,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,wicketkeeper,-33.314745,10,282.341038,66.1446,2,-9.32,-51.401633,3,-32.666667,5,192.65187,372.60773,10,-48.906467,21.21575,2,-3.302132,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>


In [232]:
player_data_master_pd = pd.DataFrame.from_dict(player_data_master, orient='index')
player_data_master_pd.reset_index(drop=False, inplace=True)
player_data_master_pd.rename(columns= {'index': 'player_id'}, inplace=True)
player_data_master_pd = player_data_master_pd.convert_dtypes()
player_data_master_pd.head(2)

,player_id,national_team_name,national_team_id,full name,born,age,batting style,fielding position,playing role,2024_pos_batting_clutch_metric,2024_matches_played,2024_neg_batting_clutch_metric,bowling style,2024_pos_bowling_clutch_metric,2024_neg_bowling_clutch_metric,also known as,height,nicknames,education
0,974087,afghanistan,40,rahmanullah gurbaz,"november 28, 2001",22y 210d,right hand bat,wicketkeeper,wicketkeeper batter,76.91326,3,-2.821053,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1,921509,afghanistan,40,ibrahim zadran,"december 12, 2001, khost",22y 196d,right hand bat,<NA>,opening batter,65.09040,3,-3.879111,right arm medium fast,NaN,NaN,<NA>,<NA>,<NA>,<NA>


In [235]:
player_data_master_updated = pd.merge(player_data_master_old, player_data_master_pd, on='player_id', how='left', suffixes=('_old', '_new'))
player_data_master_updated.head(2)

,player_id,national_team_name_old,national_team_id_old,full name_old,born_old,age_old,batting style_old,playing role_old,2024_neg_batting_clutch_metric_old,2024_matches_played_old,2024_pos_batting_clutch_metric_old,2023_pos_batting_clutch_metric,2023_matches_played,2023_neg_batting_clutch_metric,2022_neg_batting_clutch_metric,2022_matches_played,2022_pos_batting_clutch_metric,2021_neg_batting_clutch_metric,2021_matches_played,2020_neg_batting_clutch_metric,2020_matches_played,2020_pos_batting_clutch_metric,2019_neg_batting_clutch_metric,2019_matches_played,2019_pos_batting_clutch_metric,fielding position_old,2016_neg_batting_clutch_metric,2016_matches_played,2016_pos_batting_clutch_metric,2015_pos_batting_clutch_metric,2015_matches_played,2015_neg_batting_clutch_metric,2014_neg_batting_clutch_metric,2014_matches_played,2013_neg_batting_clutch_metric,2013_matches_played,2013_pos_batting_clutch_metric,2012_pos_batting_clutch_metric,2012_matches_played,2012_neg_batting_clutch_metric,2011_pos_batting_clutch_metric,2011_matches_played,2011_neg_batting_clutch_metric,bowling style_old,2021_pos_batting_clutch_metric,2018_pos_batting_clutch_metric,2018_matches_played,2018_neg_batting_clutch_metric,2024_neg_bowling_clutch_metric_old,2023_neg_bowling_clutch_metric,2022_neg_bowling_clutch_metric,2022_pos_bowling_clutch_metric,2020_neg_bowling_clutch_metric,2020_pos_bowling_clutch_metric,2018_pos_bowling_clutch_metric,2018_neg_bowling_clutch_metric,2017_neg_batting_clutch_metric,2017_matches_played,2017_pos_batting_clutch_metric,2017_pos_bowling_clutch_metric,2024_pos_bowling_clutch_metric_old,2023_pos_bowling_clutch_metric,2021_neg_bowling_clutch_metric,2021_pos_bowling_clutch_metric,2016_pos_bowling_clutch_metric,2016_neg_bowling_clutch_metric,2015_neg_bowling_clutch_metric,2015_pos_bowling_clutch_metric,2014_pos_batting_clutch_metric,2014_pos_bowling_clutch_metric,2014_neg_bowling_clutch_metric,2013_pos_bowling_clutch_metric,2012_pos_bowling_clutch_metric,2012_neg_bowling_clutch_metric,2011_neg_bowling_clutch_metric,2011_pos_bowling_clutch_metric,2019_pos_bowling_clutch_metric,education_old,also known as_old,nicknames_old,height_old,2017_neg_bowling_clutch_metric,2019_neg_bowling_clutch_metric,2009_neg_batting_clutch_metric,2009_matches_played,2009_neg_bowling_clutch_metric,2009_pos_bowling_clutch_metric,2010_pos_batting_clutch_metric,2010_matches_played,2013_neg_bowling_clutch_metric,2010_neg_batting_clutch_metric,2010_neg_bowling_clutch_metric,2010_pos_bowling_clutch_metric,2008_pos_batting_clutch_metric,2008_matches_played,2008_pos_bowling_clutch_metric,2008_neg_bowling_clutch_metric,2008_neg_batting_clutch_metric,2009_pos_batting_clutch_metric,2007_neg_batting_clutch_metric,2007_matches_played,2007_neg_bowling_clutch_metric,2007_pos_batting_clutch_metric,2007_pos_bowling_clutch_metric,other,died,national_team_name_new,national_team_id_new,full name_new,born_new,age_new,batting style_new,fielding position_new,playing role_new,2024_pos_batting_clutch_metric_new,2024_matches_played_new,2024_neg_batting_clutch_metric_new,bowling style_new,2024_pos_bowling_clutch_metric_new,2024_neg_bowling_clutch_metric_new,also known as_new,height_new,nicknames_new,education_new
0,670035,west,4,brandon alexander king,"december 16, 1994, kingston, jamaica",29y 187d,right hand bat,top order batter,-48.514704,10,505.677721,701.316028,13,-121.74086,-173.538350,17,301.907971,-35.161545,3,-36.165217,4,108.877682,-67.71203,5,41.041,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>
1,333066,west,4,johnson charles,"january 14, 1989, st lucia",35y 158d,right hand bat,wicketkeeper batter,-4

In [236]:
cols_2024 = [col for col in player_data_master_updated.columns if '2024' in col and 'new' not in col]
cols_2024

['2024_neg_batting_clutch_metric_old',
 '2024_matches_played_old',
 '2024_pos_batting_clutch_metric_old',
 '2024_neg_bowling_clutch_metric_old',
 '2024_pos_bowling_clutch_metric_old']

In [240]:
for col in cols_2024:
    new_col = col.replace('old','new')
    player_data_master_updated[new_col] = player_data_master_updated[new_col].fillna(0)
    player_data_master_updated[col] = player_data_master_updated[col] + player_data_master_updated[new_col]
    player_data_master_updated = player_data_master_updated.rename(columns={col:col.replace('_old','')})

In [246]:
pd.set_option('max.columns', None)
player_column_list = list(player_data_master_updated.columns)
for col in player_column_list:
    if '_old' in col:
        player_data_master_updated = player_data_master_updated.rename(columns={col:col.replace('_old', '')})
    elif '_new' in col:
        player_data_master_updated.drop([col], axis=1, inplace=True)

player_data_master_updated.head(2)

,player_id,national_team_name,national_team_id,full name,born,age,batting style,playing role,2024_neg_batting_clutch_metric,2024_matches_played,2024_pos_batting_clutch_metric,2023_pos_batting_clutch_metric,2023_matches_played,2023_neg_batting_clutch_metric,2022_neg_batting_clutch_metric,2022_matches_played,2022_pos_batting_clutch_metric,2021_neg_batting_clutch_metric,2021_matches_played,2020_neg_batting_clutch_metric,2020_matches_played,2020_pos_batting_clutch_metric,2019_neg_batting_clutch_metric,2019_matches_played,2019_pos_batting_clutch_metric,fielding position,2016_neg_batting_clutch_metric,2016_matches_played,2016_pos_batting_clutch_metric,2015_pos_batting_clutch_metric,2015_matches_played,2015_neg_batting_clutch_metric,2014_neg_batting_clutch_metric,2014_matches_played,2013_neg_batting_clutch_metric,2013_matches_played,2013_pos_batting_clutch_metric,2012_pos_batting_clutch_metric,2012_matches_played,2012_neg_batting_clutch_metric,2011_pos_batting_clutch_metric,2011_matches_played,2011_neg_batting_clutch_metric,bowling style,2021_pos_batting_clutch_metric,2018_pos_batting_clutch_metric,2018_matches_played,2018_neg_batting_clutch_metric,2024_neg_bowling_clutch_metric,2023_neg_bowling_clutch_metric,2022_neg_bowling_clutch_metric,2022_pos_bowling_clutch_metric,2020_neg_bowling_clutch_metric,2020_pos_bowling_clutch_metric,2018_pos_bowling_clutch_metric,2018_neg_bowling_clutch_metric,2017_neg_batting_clutch_metric,2017_matches_played,2017_pos_batting_clutch_metric,2017_pos_bowling_clutch_metric,2024_pos_bowling_clutch_metric,2023_pos_bowling_clutch_metric,2021_neg_bowling_clutch_metric,2021_pos_bowling_clutch_metric,2016_pos_bowling_clutch_metric,2016_neg_bowling_clutch_metric,2015_neg_bowling_clutch_metric,2015_pos_bowling_clutch_metric,2014_pos_batting_clutch_metric,2014_pos_bowling_clutch_metric,2014_neg_bowling_clutch_metric,2013_pos_bowling_clutch_metric,2012_pos_bowling_clutch_metric,2012_neg_bowling_clutch_metric,2011_neg_bowling_clutch_metric,2011_pos_bowling_clutch_metric,2019_pos_bowling_clutch_metric,education,also known as,nicknames,height,2017_neg_bowling_clutch_metric,2019_neg_bowling_clutch_metric,2009_neg_batting_clutch_metric,2009_matches_played,2009_neg_bowling_clutch_metric,2009_pos_bowling_clutch_metric,2010_pos_batting_clutch_metric,2010_matches_played,2013_neg_bowling_clutch_metric,2010_neg_batting_clutch_metric,2010_neg_bowling_clutch_metric,2010_pos_bowling_clutch_metric,2008_pos_batting_clutch_metric,2008_matches_played,2008_pos_bowling_clutch_metric,2008_neg_bowling_clutch_metric,2008_neg_batting_clutch_metric,2009_pos_batting_clutch_metric,2007_neg_batting_clutch_metric,2007_matches_played,2007_neg_bowling_clutch_metric,2007_pos_batting_clutch_metric,2007_pos_bowling_clutch_metric,other,died
0,670035,west,4,brandon alexander king,"december 16, 1994, kingston, jamaica",29y 187d,right hand bat,top order batter,-48.514704,10,505.677721,701.316028,13,-121.74086,-173.538350,17,301.907971,-35.161545,3,-36.165217,4,108.877682,-67.71203,5,41.041,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>
1,333066,west,4,johnson charles,"january 14, 1989, st lucia",35y 158d,right hand bat,wicketkeeper batter,-47.562383,11,310.139647,293.131878,7,-47.39507,-35.783941,4,66.991050,NaN,<NA>,NaN,<NA>,NaN,NaN,<NA>,NaN,wicketkeeper,-33.314745,10,282.341038,66.1446,2,-9.32,-51.401633,3,-32.666667,5,192.65187,372.60773,10,-48.906467,21.21575,2,-3.302132,<NA>,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,<NA>


In [247]:
# player_data_master_pd.to_csv('player_data_master_t20_wc.csv', index=False)
player_data_master_updated.to_csv('player_data_master_t20_wc_updated.csv', index=False)

Debugger

In [51]:
skipped_matches_pd = pd.DataFrame(skipped_matches)
skipped_matches_pd.to_csv('skipped_matches_t20_wc.csv', index=False)

In [52]:
skipped_matches_pd.head()

,date,venue_link,venue_name,match_url,match_id
0,"Jun 19, 2024",/cricket-grounds/happy-valley-ground-episkopi-...,Episkopi,/series/cyprus-vs-estonia-2024-1438070/cyprus-...,2707
1,"Jun 19, 2024",/cricket-grounds/happy-valley-ground-episkopi-...,Episkopi,/series/cyprus-vs-estonia-2024-1438070/cyprus-...,2706
2,"Jun 18, 2024",/cricket-grounds/happy-valley-ground-episkopi-...,Episkopi,/series/cyprus-vs-estonia-2024-1438070/cyprus-...,2705
3,"Jun 18, 2024",/cricket-grounds/happy-valley-ground-episkopi-...,Episkopi,/series/cyprus-vs-estonia-2024-1438070/cyprus-...,2704
4,"Jun 17, 2024",/cricket-grounds/happy-valley-ground-episkopi-...,Episkopi,/series/cyprus-vs-estonia-2024-1438070/cyprus-...,2701


In [55]:
spl_matches_pd = pd.DataFrame(spl_matches)
spl_matches_pd.to_csv('spl_matches_t20_wc.csv', index=False)

In [56]:
spl_matches_pd.head()

,id,url,stage,error,match_details_list
0,348133,https://www.espncricinfo.com/series/icc-men-s-...,3,could not convert string to float: '',"[{'date': 'Jun 19, 2024', 'venue_link': '/cric..."
1,303423,https://www.espncricinfo.com/series/icc-men-s-...,3,could not convert string to float: '',"[{'date': 'Jun 19, 2024', 'venue_link': '/cric..."
2,794707,https://www.espncricinfo.com/cricketers/pratis...,7,'bowling style',"[{'date': 'Jun 19, 2024', 'venue_link': '/cric..."
3,359746,https://www.espncricinfo.com/cricketers/parvee...,7,'bowling style',"[{'date': 'Jun 19, 2024', 'venue_link': '/cric..."
4,348133,https://www.espncricinfo.com/series/usa-vs-can...,7,'bowling style',"[{'date': 'Jun 19, 2024', 'venue_link': '/cric..."


In [ ]:
driver.close()

Player Wise Stats

In [7]:
tournament_suburls = {2024:'icc-men-s-t20-world-cup-2024-15946',
                   2022: 'icc-men-s-t20-world-cup-2022-23-14450',
                   2021: 'icc-men-s-t20-world-cup-2021-22-14024',
                   2016: 'world-t20-2015-16-10291',
                   2014: 'world-t20-2013-14-8083',
                   2012: 'icc-world-twenty20-2012-13-6856',
                   2010: 'icc-world-twenty20-2010-5195',
                   2009: 'icc-world-twenty20-2009-4162',
                   2007: 'icc-world-twenty20-2007-08-3115'}

In [8]:
batting_urls = {}
bowling_urls = {}
for year, suburl in tournament_suburls.items():
    batting_urls[year] = 'https://www.espncricinfo.com/records/tournament/batting-most-runs-career/' + suburl
    bowling_urls[year] = 'https://www.espncricinfo.com/records/tournament/bowling-most-wickets-career/' + suburl

In [10]:
bowling_urls_sample = {}
bowling_urls_sample[2024] = bowling_urls[2024]

batting_urls_sample = {}
batting_urls_sample[2024] = batting_urls[2024]

In [11]:
top_bowlers_master = {}

for season, url in bowling_urls_sample.items():
    driver.get(url)
    print(season)
    
    html = driver.page_source.encode('utf-8')
    page_num = 0
    content = driver.page_source
    page_soup = soup(driver.page_source, 'lxml')

    table = page_soup.findAll('td', {'class':'ds-min-w-max'})[15:]
    labels = page_soup.findAll('td', {'class':'ds-min-w-max'})[:15]
    
    all_labels = []
    for i in range(15):
        all_labels.append(labels[i].text)
    
    top_bowlers = []
    count = 0

    for td_count in range(len(table)):
        if(td_count%15==0):
            if (td_count!=0):
                json_obj['team'] = json_obj['Player'].split(' (')[1].replace(')', '')
                json_obj['bowler_surname'] = json_obj['Player'].split(' ')[-2]
                top_bowlers.append(json_obj)
            json_obj = {}
            json_obj['year'] = season
            json_obj['player_url'] = table[td_count].span.a['href']
            json_obj['player_id'] = json_obj['player_url'].split('-')[-1]           
            
        json_obj[all_labels[td_count%15]] = table[td_count].text                 

    top_bowlers_pd = pd.DataFrame.from_dict(top_bowlers)
    top_bowlers_master[season] = top_bowlers_pd

2024


In [12]:
top_bowlers_pd.head(2)

,year,player_url,player_id,Player,Span,Mat,Inns,Balls,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5,team,bowler_surname
0,2024,/cricketers/fazalhaq-farooqi-974175,974175,Fazalhaq Farooqi (AFG),2024-2024,8,8,152,25.2,0,160,17,5/9,9.41,6.31,8.94,1,1,AFG,Farooqi
1,2024,/cricketers/arshdeep-singh-1125976,1125976,Arshdeep Singh (IND),2024-2024,8,8,180,30.0,0,215,17,4/9,12.64,7.16,10.58,1,0,IND,Singh


In [13]:
top_batters_master = {}

for season, url in batting_urls_sample.items():
    driver.get(url)
    print(season)
    
    html = driver.page_source.encode('utf-8')
    page_num = 0
    content = driver.page_source
    page_soup = soup(driver.page_source, 'lxml')

    table = page_soup.findAll('td', {'class':'ds-min-w-max'})[15:]
    labels = page_soup.findAll('td', {'class':'ds-min-w-max'})[:15]
    
    all_labels = []
    for i in range(15):
        all_labels.append(labels[i].text)
    
    top_batters = []
    count = 0

    for td_count in range(len(table)):
        if(td_count%15==0):
            if (td_count!=0):
                json_obj['team'] = json_obj['Player'].split(' (')[1].replace(')', '')
                json_obj['batter_surname'] = json_obj['Player'].split(' ')[-2]
                top_batters.append(json_obj)
            json_obj = {}
            json_obj['year'] = season
            json_obj['player_url'] = table[td_count].span.a['href']
            json_obj['player_id'] = json_obj['player_url'].split('-')[-1]           
            
        json_obj[all_labels[td_count%15]] = table[td_count].text                 

    top_batters_pd = pd.DataFrame.from_dict(top_batters)
    top_batters_master[season] = top_batters_pd

2024


In [14]:
top_batters_pd.head(2)

,year,player_url,player_id,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s,team,batter_surname
0,2024,/cricketers/rahmanullah-gurbaz-974087,974087,Rahmanullah Gurbaz (AFG),2024-2024,8,8,0,281,80,35.12,226,124.33,0,3,2,18,16,AFG,Gurbaz
1,2024,/cricketers/rohit-sharma-34102,34102,RG Sharma (IND),2024-2024,8,8,1,257,92,36.71,164,156.70,0,3,0,24,15,IND,Sharma


Taking only the top 15 batsmen/bowlers per season and adding their clutch metric

In [17]:
player_data_master = pd.read_csv('player_data_master_t20_wc.csv')
player_data_master = player_data_master.convert_dtypes()
player_data_master['player_id'] = player_data_master['player_id'].astype(str)

In [18]:
first_time = True
for year, item in top_batters_master.items():
    print(year)
    pd_data = item.head(20).copy().reset_index()
    pd_data = pd_data.convert_dtypes()
    pd_data = item.head(15).copy().reset_index()
    year_metrics_columns = [column_name for column_name in list(player_data_master.columns) if (str(year) in column_name) and ('bowling' not in column_name)]
    year_metrics_columns.append('player_id')    
    pd_data['player_id'] = pd_data['player_id'].astype(str)
    pd_data = pd.merge(pd_data, player_data_master[year_metrics_columns], on='player_id', how='left')
    pd_data = pd_data.rename(columns=lambda x: x.replace(str(year), 'year') if str(year) in x else x) 
    pd_data['year'] = year

    if first_time:
        top_batters_pd = pd_data
        first_time = False
    else:
        top_batters_pd = pd.concat([top_batters_pd, pd_data])

top_batters_pd.rename(columns={'index':'rank'}, inplace=True)
top_batters_pd['rank'] = top_batters_pd['rank'] + 1
top_batters_pd.reset_index(drop=True, inplace=True)

2024


In [19]:
first_time = True
for year, item in top_bowlers_master.items():
    print(year)
    pd_data = item.head(20).copy().reset_index()
    pd_data = pd_data.convert_dtypes()
    year_metrics_columns = [column_name for column_name in list(player_data_master.columns) if (str(year) in column_name) and ('batting' not in column_name)]
    year_metrics_columns.append('player_id')
    pd_data['player_id'] = pd_data['player_id'].astype(str)
    pd_data = pd.merge(pd_data, player_data_master[year_metrics_columns], on='player_id', how='left')
    pd_data = pd_data.rename(columns=lambda x: x.replace(str(year), 'year') if str(year) in x else x) 
    pd_data['year'] = year

    if first_time == True:
        top_bowlers_pd = pd_data
        first_time = False
    else:
        top_bowlers_pd = pd.concat([top_bowlers_pd, pd_data])

top_bowlers_pd.rename(columns={'index':'rank'}, inplace=True)
top_bowlers_pd['rank'] = top_bowlers_pd['rank'] + 1
top_bowlers_pd.reset_index(drop=True, inplace=True)

2024


In [20]:
top_bowlers_pd.to_csv('top_bowlers_t20_wc_v3.csv', index = False)
top_batters_pd.to_csv('top_batters_t20_wc_v3.csv', index = False)

In [21]:
driver.close()